In [2]:
import json
from pprint import pprint
import pandas as pd
import os
import re
import regex
import time

In [3]:
base_path="/Users/sijiawu/Work/Refs Danae/Thesis/Data/"

In [4]:
# Merged=pd.read_excel(base_path+'/Combined/AER_M_sco_du.xlsx')
# Merged.loc[Merged['journal']=='The American Economic Review','journal']='AER'

JOURNALS= [
    'AER', 
    'JPE', 
    'ECTA', 
    'RES', 
    'QJE'
]
#read in all processed masterlists
j_data=pd.DataFrame()
for i in JOURNALS:
    j_data=pd.concat([pd.read_excel('/Users/sijiawu/Work/Refs Danae/Thesis/Data/Combined/'+i+'_M_sco_du.xlsx'), j_data], ignore_index=True)
#Create a batch file

j_data=j_data[j_data.duplicated()==False].reset_index().drop('index', axis=1)

# Replace the journal names with Acronyms
j_data.loc[j_data['journal']=="Econometrica",'journal']='econometrica'
# j_data.loc[j_data['journal']=='The Quarterly Journal of Economics','journal']='quarterly journal of economics'
# j_data.loc[j_data['journal']=='The Review of Economic Studies','journal']='review of economic studies'
# j_data.loc[j_data['journal']=='Journal of Political Economy','journal']='journal of political economy'
# j_data.loc[j_data['journal']=='The American Economic Review','journal']='american economic review'

#some corrections to the issue
j_data.loc[j_data["number"]=="2023-03-04 00:00:00","number"]="3--4"
j_data.loc[j_data["number"]=="4-5","number"]="4--5"
j_data.loc[j_data["number"]=="1-2","number"]="1--2"

j_data["ID"]=j_data["URL"].str.split("/").str[-1]

j_data.journal.unique()
Merged=j_data

In [5]:
def open_json(filepath):
    with open(filepath) as json_data:
        d = json.load(json_data)
        json_data.close()
        return d    

In [6]:
Merged.shape

(62262, 31)

In [7]:
l_check=[]
av=0
js=0
issue=[]
js_2=0
js_3=0
js_4=0
js_5=0
references={}
manu=[]
manu2=[]
ex1=[]
ex2=[]
keys=[]
jour=[]
alt=[]
alt2=[]
alt3=[]
nj=[]
probs=[]
eds=[]
count=0
book=0
al=[]
for i in Merged.index:
    count=count+1
    paper_id=Merged.loc[i,"ID"]
    p_ref={"refs":[], "year":Merged.loc[i, "year"]} 
    filename=base_path+"gpt/"+Merged.loc[i,"ID"]+"_chatgpt.json"
#     if paper_id!="45217222":
#         continue
    if os.path.exists(filename):
        ref_count=0
        a=open_json(filename)
        print(paper_id)
        content=a[paper_id]['choices'][0]['message']['content']
        if content[0]=="{":
            js=js+1
            ref_split=re.findall("{[^{}]*}", content)
            for k in ref_split:
                ref_single=k.split("\n")
#                 print(k)
                if len(ref_single)<5:
                    continue
                
                i_ref={}
               
                for r in ref_single:
                    reg_groups=re.search('\s+"([^:]*?)": "(.*)"(|,)',r)
                    if reg_groups is not None:
                        a=reg_groups.groups()
                        i_ref[a[0]]=a[1]
                        keys.append(a[0])
                
                
                if set(["full_reference", "authors", "year", "title"]).issubset(list(i_ref.keys())):
                    if ((i_ref["title"]=="NA")&(i_ref["year"]=="NA"))|(i_ref["title"]=="NA")&(i_ref["authors"]=="NA")|(i_ref["full_reference"]=="NA"):
                        continue
                    if i_ref["full_reference"]=="NA":
                        continue
                        
                    lx=re.sub(r"[‘’]","'", i_ref["title"].lower())
                    lx=re.sub(r"[“”]",'"', lx)
                    lx=re.sub(r"''",'"',lx)
                    lx=re.sub(r'""','"',lx)
                    lx=re.sub(r'\'"\'','"',lx)
                    lx=re.sub(r"(?<=\D)(- )(?=\D)", "", lx)
                    
                    rx=re.sub(r"[“”]",'"',i_ref["full_reference"].lower())
                    rx=re.sub(r"[‘’]","'", rx)
                    rx=re.sub(r"''",'"',rx)
                    rx=re.sub(r'""','"',rx)
                    rx=re.sub(r'\'"\'','"',rx)
                    rx=re.sub(r"(?<=\D)(- )(?=\D)", "", rx)
                    
                    
                    lx_x=re.sub("[()\[\]'*]",".",lx)
                    lx_x=regex.sub(r'\\(\d)','\1',lx_x)
                    rx_x=re.sub("[\[\]]",".",rx)
                    
                    l_check.append(rx)
                    
                    if regex.search("MANUSCRIPT RECEIVED|CO-EDITOR.*HANDLED THIS MANUSCRIPT".lower(),i_ref["full_reference"].lower()) is not None:
                        manu.append(i_ref["full_reference"])
                        continue
                        
                    if (regex.search("MANUSCRIPT RECEIVED|CO-EDITOR.*HANDLED THIS MANUSCRIPT".lower(),i_ref["title"].lower()) is not None):
                        manu2.append(i_ref["title"])
                        continue
                    i_ref["status"]=True
                    if (lx not in rx):
                        r = regex.compile('(%s){e<=5}' % lx_x)
                        if r.search(rx_x) is None:
                            print("***** issue *****")
                            print(lx)
                            print(rx)
                            print(lx_x)
                            print(rx_x)
                            print("***** issue *****")
                            i_ref['status']=False
                    t1=time.time()

                    reg1=r'(?<!".+)([^()":0-9]+)[\(]*(\d+\D*?|na|forthcoming|undated)[\)]*[:. ]+"(.+)"[\'\"-+,\. ]+([^"\d]+?)(vol.*|\d.*)(\[.+\]|)'
                    reg2=r'(?<!".+)([^()":0-9]+)[\(]*(\d{2,5}\D{0,2}?|na|forthcoming|undated)[:. \)]+(".+|)'
                    reg3=r'(\D+).(\d+[^0-9.]*?|na).([^.]*).(.+?)(\d+):(.*)'
                    reg4=r'(?<!".+)([^()":0-9]+)[:. ]+"(.+)"[\'\"-+,\. ]+([^"\d]+)(vol.*|\d*)'
                    if regex.search(reg1,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg1,rx).groups()[3]
#                         print("found")
#                         print(rx)
                        i_ref['regex']={"pattern":reg1, "type":1}
                        chap=regex.search("^chap|^chapter |^ch. | chap.|chapter", i_ref["alt_j"].strip())
                        ed=regex.search('ed\. | eds\. |edited by|editor| ed\.|\(ed\.|eds\.\)', i_ref["alt_j"].strip())
                        if "ed." in rx or "eds." in rx or "edited by" in rx or "editor" in rx:
                            book+=1
                        has_in=regex.search(r"^in .+",i_ref["alt_j"].strip())
                        http=regex.search(r'http:|https:', rx)
                        accessed_at=regex.search(r'accessed at http',rx)
                        dis_paper=regex.search("discussion paper",rx)
                        work_paper=regex.search("working paper", rx)
                        un_pub=regex.search("unpublished .*", i_ref["alt_j"])
                        report=regex.search("^report", i_ref["alt_j"].strip())
#                         if chap is not None or ed is not None or has_in is not None:
#                             print(chap is not None)
#                             print(ed is not None)
#                             print(has_in is not None)
                            
#                         else:
#                             print(i_ref["alt_j"])
#                         if http is not None:
#                             print(http is not None)
#                             print(rx)
                        jour.append(i_ref["alt_j"].strip())
                    
                    elif regex.search(reg2,rx) is not None:
#                         print("found likely chicago style pattern but not journal reference pattern")
                        i_ref['regex']={"pattern":reg2, "type":2, "match_target":rx}
#                         print(regex.search(reg2,rx).groups())
                        alt.append(rx)
                    elif regex.search(reg4,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg4,rx).groups()[2]
                        i_ref['regex']={"pattern":reg4, "type":4, "match_target":rx}

                        alt3.append(rx)
                    elif regex.search(reg3,rx) is not None:
                        i_ref["alt_j"]= regex.search(reg3,rx).groups()[4]
#                         print("found alternative style pattern")
                        i_ref['regex']={"pattern":reg3, "type":3, "match_target":rx}
                        alt2.append(rx)
                    

                    else:
                        nj.append(rx)
                        print("not found")
                        i_ref['regex']={"pattern":None, "type":None, "match_target":rx}
#                         print(lx_x)
                        print(rx)
                        print("\n")
                        probs.append(paper_id)
#                         print()
                    t2=time.time()
#                     print(t2-t1)
                              
                p_ref["refs"].append(i_ref)

        elif content[0:15]=="1. Reference 1:":
            js_2=js_2+1
        elif content[0:13]=="1. Reference:":
            js_3=js_3+1
        elif content[0:5]=="1. \n{":
            js_4=js_4+1
        elif content[0:5]=="1. \n-":
            js_5=js_5+1
        else:
            print(content[:400])
            print(paper_id)
            print('\n')
        av=av+1
        references[paper_id]=p_ref
#         print("\n\n")
        

qjaa018
qjaa017
qjaa003
not found
€z0z jequiaides gq uo jesn seueigi] umol eden jo aussenlup aq sozlzzs/blel/e/se | /elolue/elb/woo'dno'silwepece//:sdyy wo pepeojumoqg


qjz043
not found
€zz0z jequiaides gq uo jesn seueigi] umol eden jo aussenlu aq s9/s69s/es1 l/z/se | /elolue/elb/woo'dno'slwepece//:sdyjy wo pepeojumoq


qjz040
qjz035
qjz033
qjz032
not found
kleven, henrik j., 'bunching," annual review of economics, 8 (2016), 485-464.


***** issue *****
taxation and migration: evidence and policy implications
kleven, henrik j., and esben a. schultz, "estimating taxable income responses using danish tax reforms," american economic journal: economic policy, 6 (2014), 271-801.
taxation and migration: evidence and policy implications
kleven, henrik j., and esben a. schultz, "estimating taxable income responses using danish tax reforms," american economic journal: economic policy, 6 (2014), 271-801.
***** issue *****
not found
na, 'taxation of intergenerational transfers and wealth," in ha

26372716
26372718
not found
the deposits channel of monetary policy 1875


26372719
not found
hela halsningland, "rather mayor than parliamentarian (hellre kommu-nalrad ar riksdgsman)," 2010, http://www.helahalsingland.se/halsingland/soderhamn/hellre-kommunalrad-an-riksdagsman.


not found
sveriges radio, "would rather be mayor than parliamentarian [blir hellre kommunalrad 4n riksdagsman]," 2006, http://sverigesradio.se/sida/artikel.aspx? programid=161&artikel=973385.


not found
zetterberg, hans, "during olof palme's time—and afterwards (under olof palmes tid och efterat)," 1988, http://zetterberg.org/press/svd/sd880229.htm.


26372720
not found
1966 quarterly journal of economics


26372721
26372722
26372699
26372701
26372702
26372703
26372704
26372707
26495145
26495146
26495148
26495150
not found
brac, "stories of targeting: process documentation of selecting the ultra poor for cfpr/tup programme, brac red cfpr-tup wp series no. 1," 2004.


not found
brac, "annual report," 2014, dha

23251989
23251990
23251993
not found
rogerson, richard, "structural transformation and the deterioration of european labor market outcomes,"journal of political economy, 116 (2008), 235-259


23251994
23251996
not found
caliendo, marco, konstantinos tatsiramos, and arne uhlendorff, "benefit duration, unemployment duration and job match quality: a regression-discontinuity approach, iza discussion paper 4670, 2009


23251997
23251998
23251999
23252000
23252002
41337205
not found
fehr, ernst, and simon gachter, "fairness and retaliation—the economics of reciprocity,"journal of economic perspectives, 14(2000), 159-181.


41337207
not found
cet ariel, modeling bounded rationality (cambridge, mass.: mit press).


41337208
41337209
41337210
41337211
41337213
41337214
41337215
41337175
41337178
41337180
not found
aoki, masahiko, hugh patrick and paul sheard,"the japanese main bank system: an introductory overview," in the japanese main bank system: its relevance for developing and transforming

25098909
not found
goldberg, pinelopi k., and michael m. knetter, "goods prices and exchange rates: what have we learned? journal of economic literature, 35 (3) (1997), 1243—1272.


25098910
25098911
not found
assael, henry, consumer behavior and marketing action, 2\"4 ed. (boston: kent publishing company, 1984)


25098913
not found
reinikka, r., and j. svensson, "fighting corruption to improve schooling: evidence from a newspaper campaign in uganda, journal of the european economic association, 3(2—3) (2005), 259-267.


25098915
25098916
not found
moorhouse, j.,"is tullock correct about sunday closing laws?," public choice, 42 (1984), 197-203


25098898
25098900
not found
grofman, bernard, lisa handley, and richard niemi, minority representation and the quest for voting equality (new york: cambridge university press)


25098901
not found
harvey, campbell r., and akhtar r. siddique,"conditional skewness in asset pricing," the journal of finance, 55 (2000), 1263-1295.


not found
kraus,

25098685
not found
imf, annual report on exchange arrangements and exchange restrictions (washington, dc: various editions).


25053937
25053939
25053941
not found
aoa \\iaee daniel, labor demand (princeton, nj: princeton university press, 1).


25053942
25053943
25053945
not found
moretti, enrico. "workers' education, spillovers and productivity: evidence from plant-level production functions." 2002.


25053946
25053923
not found
aumann, robert j., and michael maschler with the collaboration of r. stearns, repeated games with incomplete information (cambridge, ma: mit press, 1 .


not found
cripps, martin, and jonathon thomas, "some asymptotic results in discounted repeated games with incomplete information," 2001, http://www.olin.wustl.edu/faculty/cripps/nash2001sent.pdf.


25053924
25053925
25053926
25053927
25053928
25053929
25053930
25053931
25053932
not found
u.s. department of commerce, bureau of the census, contiguous county file (washington, dc: u. s. department of commerce, b

...

not found
duncan foley and miguel sidrauski, monetary and fiscal policy in a growing economy, new york 1971


not found
don patinkin, money, interest, and prices, new york 1965


1802510
1802511
1802512
1802513
not found
r. h. snape, '"forward exchange and futures markets," in lan a. mcdougall and r. h. snape, eds., studies in international economics, monash conference papers, amsterdam 1970.


1802514
1802515
not found
paul a. david et al., reckoning with slavery: a critical study in the quantitative history of american negro slavery, new york 1976


not found
robert w. fogel and stanley l. engerman, 7ime on the cross, vol. 1, boston 1974


1802516
1802517
not found
g. wright, '"an econometric study of cotton production and trade, 1830-1860," rev. econ. statist., may 1971, 53, 111-20.


1802518
not found
na, '"cotton, corn and risk in the nineteenth century," j. econ. hist., sept. 1975, 35, 526-51.


1802519
not found
m. i. kamien and n. l. schwartz, '"the optimal resource-capital rat

not found
"an econometric analysis of the chow and megdal: inflation and unemployment 453 relation of monetary variables to the behavior of prices and unemployment," in otto eckstein, ed., the econometrics of price determination, washington 1972, 166-83.


1805280
1805281
1805282
not found
j. w. pratt, '"risk aversion in the small and in the large," econometrica, jan./apr. 1964, 32, 122-36.


1805283
not found
j. e. anderson, "the social cost of input distortions: a comment and a generalization,' amer. econ. rev., mar. 1976, 66, 235-38.


not found
r. schmalensee, "consumer's surplus and producer's goods,'"? amer. econ. rev., sept. 1971, 6/7, 682-87.


1816653
1816654
1816655
1816656
not found
g. a. akerlof, '"the market for 'lemons': quality, uncertainty and the market mechanism," quart. j. econ., aug. 1970, 84, 488-500.


not found
r. cassady, jr., '"negotiated price making in mexican traditional markets," amer. indigena, 1968, 38, 51-79.


not found
f. khuri, '"the etiquette of barg

not found
"competitive optimal responses to signals: an analysis of efficiency and distribution," j. econ. theory, mar. 1974, 7, 296-332.


1828077
1828078
1828079
1828080
1828081
1828082
not found
'"*pareto-optimal redistribution': a perspective," finanzarchiv, 1975, 33, 237-71.


not found
g. daly and f. giertz, '"welfare economics and welfare reform," amer. econ. rev., mar. 1972, 62, 131-38.


not found
g. e. peterson, '"welfare, workfare and pareto optimality," publ. finance quart., july 1973, 7, 323-38.


1828083
1828084
not found
y. ben-porath, '"the production of human capital and the life cycle of earning," j. polit. econ., aug. 1967, 75, 352-65.


1828085
1828086
1828087
1813387
1813388
1813389
1813392
not found
r. b. fernandez, '"short run dynamics of output and prices," unpublished doctoral dissertation, univ. chicago 1975.


not found
international monetary fund, international financial statistics (ifs) washington, various years.


not found
instituto nacional de estadistic

***** issue *****
an exploration in the theory of optimum income taxation
james mirrlees, '"an exploration in the distribution of income and wealth 85 theory of optimum income taxation,"' rev. econ. stud., apr. 1971, 38, 179-208.
an exploration in the theory of optimum income taxation
james mirrlees, '"an exploration in the distribution of income and wealth 85 theory of optimum income taxation,"' rev. econ. stud., apr. 1971, 38, 179-208.
***** issue *****
1815888
not found
pamela christoffel, a compilation of federal programs financing postsecondary education. national commission on the financing of postsecondary education, washington 1973


not found
carol van alstyne and sharon l. coldren, the costs of implementing federally mandated social programs at colleges and universities, policy analysis service, american council on education, june,1976


not found
carnegie council on policy studies in higher education, economic problems confronting higher education, san francisco 1975


not f

not found
h. w. guthrie, \"who moonlights and why,\" illinois business review, mar. 1965, 22, 6-8.


not found
j. tobin, \"estimation of relationships for limited dependent variables,\" monthly labor review, feb. 1969, 92, 28-31.


not found
na, \"teachers in the moonlight,\" metrica, jan. 1958, 26, 24-36.


1828165
1. J. S. DUESENBERRY, INCOME, SAVING, AND THE THEORY OF CONSUMER BEHAVIOR, CAMBRIDGE, MASS. 1949.
   - Authors: J. S. DUESENBERRY
   - Year: 1949
   - Title: INCOME, SAVING, AND THE THEORY OF CONSUMER BEHAVIOR
   - Month: NA
   - Publisher: CAMBRIDGE
   - Pages: NA
   - Full Reference (Chicago style): DUESENBERRY, J. S. INCOME, SAVING, AND THE THEORY OF CONSUMER BEHAVIOR. CAMBRIDGE, MASS., 1949.


1828165


1828167
not found
j. bhagwati and t. n. srinivasan, "on re-analyzing the harris-todaro model: policy rankings in the case of sector-specific sticky wages,' amer. econ. rev., june 1974, 64, 502-08.


not found
h. g. johnson, "optimal trade intervention."


1828170
not fou

In [8]:
lx_x

'a training system for graduate student instructors of introductory economics at the university of minnesota'

In [741]:
check='FORBES, SILKE J., AND MARA LEDERMAN, “ADAPTATION AND VERTICAL INTEGRATION IN THE AIRLINE INDUSTRY,” AMERICAN ECONOMIC REVIEW, 99 (2009), 1831-1849.'
reg4=r'(?<!".+)([^()":0-9]+)[:. ]+"(.+)"[\'\"-+,\. ]+([^"\d]+)(vol.*|\d*)'
temp=regex.search(reg4,check)


In [735]:
references["26372510"]

{'refs': [{'authors': 'AGHION, PHILIPPE, RACHEL GRIFFITH, AND PETER HOWITT',
   'year': '2006',
   'title': 'VERTICAL INTEGRATION AND COMPETITION',
   'month': 'NA',
   'publisher': 'AMERICAN ECONOMIC REVIEW PAPERS AND PROCEEDINGS',
   'pages': '97-102',
   'full_reference': 'AGHION, PHILIPPE, RACHEL GRIFFITH, AND PETER HOWITT, “VERTICAL INTEGRATION AND COMPETITION,” AMERICAN ECONOMIC REVIEW PAPERS AND PROCEEDINGS, 96 (2006), 97-102.',
   'status': True,
   'alt_j': 'american economic review papers and proceedings, ',
   'regex': {'pattern': '(?<!".+)([^()":0-9]+)[:. ]+"(.+)"[\\\'\\"-+,\\. ]+([^"\\d]+)(vol.*|\\d*)',
    'type': 4}},
  {'authors': 'AGHION, PHILIPPE, AND JEAN TIROLE',
   'year': '1994',
   'title': 'THE MANAGEMENT OF INNOVATION',
   'month': 'NA',
   'publisher': 'QUARTERLY JOURNAL OF ECONOMICS',
   'pages': '1185-1209',
   'full_reference': 'AGHION, PHILIPPE, AND JEAN TIROLE, “THE MANAGEMENT OF INNOVATION,” QUARTERLY JOURNAL OF ECONOMICS, 109 (1994), 1185-1209.',
   'st

In [9]:
e_list=list(set(eds))
e_list.sort()
len(e_list)
e_list

[]

In [10]:
len(alt2)

24

In [11]:
len(alt3)

97327

In [12]:
book

5316

In [13]:
len(alt)

75032

In [14]:
len(nj)

8456

In [15]:
(8456+172)/(len(jour) +len(alt)+len(alt3)+len(nj))

0.02960340089139587

In [16]:
print(len(probs))
p_list=list(set(probs))

8456


In [17]:
p_list[0]

'1912975'

In [18]:
j_data.columns

Index(['issue_url', 'ISSN', 'URL', 'journal', 'number', 'publisher', 'title',
       'urldate', 'volume', 'year', 'abstract', 'author', 'pages',
       'reviewed-author', 'uploaded', 'content_type', 'author_split',
       'title_10', 'type', 'authorsSCO', 'titleSCO', 'journalSCO', 'DOI',
       'affiliations', 'abstractSCO', 'citations', 'document type',
       'index keywords', 'author keywords', 'document_type', 'ID'],
      dtype='object')

In [20]:
tidle=[]
ks=[]
idle=[]
journal=[]
halu=[]
for i in references.keys():
    tidle.append(len(references[i]["refs"]))
    idle.append(probs.count(i))
    ks.append(i)
    hal=0
    for k in references[i]["refs"]:
        if "status" not in k.keys():
            continue
        if k['status']==False:
            hal+=1
    halu.append(hal)
    journal.append(j_data[j_data.ID==i]['journal'].values[0])

In [21]:
data={"ID": ks, "total": tidle,"probs": idle, "journal": journal}
out=pd.DataFrame.from_dict(data)

In [22]:
out.sort_values(by=["probs"],ascending=False)


,ID,total,probs,journal
17006,1815878,33,28,The American Economic Review
13490,3083286,28,25,The American Economic Review
14925,2006701,67,24,The American Economic Review
620,2586883,65,22,The Quarterly Journal of Economics
13517,3083352,62,22,The American Economic Review
...,...,...,...,...
6298,1911194,14,0,econometrica
6299,1911195,8,0,econometrica
6300,1911196,26,0,econometrica
6301,1911197,21,0,econometrica


In [23]:
out['rat']=out.probs/out.total

In [24]:
out[out.rat>0.1]

,ID,total,probs,journal,rat
149,26372509,16,3,The Quarterly Journal of Economics,0.187500
322,25098901,36,4,The Quarterly Journal of Economics,0.111111
359,25098842,33,5,The Quarterly Journal of Economics,0.151515
421,25098750,27,3,The Quarterly Journal of Economics,0.111111
455,25098693,45,5,The Quarterly Journal of Economics,0.111111
...,...,...,...,...,...
17131,1817211,8,2,The American Economic Review,0.250000
17133,1817213,18,5,The American Economic Review,0.277778
17137,1817220,16,4,The American Economic Review,0.250000
17138,1817221,7,1,The American Economic Review,0.142857


In [25]:
len(cont.keys())

NameError: name 'cont' is not defined

In [713]:
1200+846+756+617+556

3975

In [26]:
out[out.rat>0]["journal"].value_counts()

journal
The American Economic Review          1200
econometrica                           846
Journal of Political Economy           759
The Review of Economic Studies         617
The Quarterly Journal of Economics     556
Name: count, dtype: int64

In [715]:
2705+1992+1985+1536+886-3975

5129

In [27]:
out["journal"].value_counts()

journal
The American Economic Review          6103
econometrica                          3986
Journal of Political Economy          2802
The Review of Economic Studies        2530
The Quarterly Journal of Economics    1721
Name: count, dtype: int64

In [716]:
17142-5129-3975

8038

In [590]:
idle_sort=dict(sorted(idle.items(), key=lambda item: item[1], reverse=True))
set(idle.values())

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 65}

In [536]:
len(references.keys())

17142

In [1]:
count

NameError: name 'count' is not defined

In [272]:
j_list=list(set(jour))
len(j_list)

14954

In [273]:
len(jour)

110638

In [297]:
for i in range(len(j_list)):
    j_list[i]=regex.sub("^\[eee ","ieee",j_list[i])
    j_list[i]=regex.sub("^/n","in",j_list[i])
#     j_list[i]=regex.sub("^/pe.$|\].pe.|\[j.p.e.|\[.pe.|\[.p.e.|\[.p-e.|\[.-pe.|\[. pe.|\[,pe.|/. pe.|/.,pe.|/.-p.e.|/.-pe.|/.pe|^\[pe|\[ pe.|\[-p.e.|\[-pe.|/\[.p.e|/.re.|/.pe.|/.p.e.|/.p-e.","j.p.e.",j_list[i])
    j_list[i]=regex.sub("a-e\.r\.|a\./\.r\.|a\.e\.c\. research and development report, anl-|a\.e\.lr\.|a\.elr\.|a\.e\.r\. \'|a\.e\.r\. papers and proc.|a\.e\.r\., lvii, no.|a.e_r.|a\.el\.r\.|a\.er\.|a\.e£\.r\.|a\.f\.r\.|a\.l\.r\.|a\.£\.r\.","a.e.r.", j_list[i])
    for k in rep_names.keys():
        if j_list[i] in rep_names[k]:
            j_list[i]=k
            break
book_chaps=0
h_link=0
man=0
mim=0
for i in range(len(j_list)):
    if j_list[i][:3]=="in ":
        print(j_list[i])
        book_chaps+=1
        continue
        
    if j_list[i][:9]=="accessed ":
        h_link+=1
        continue
        
    if j_list[i][:10]=="available ":
        h_link+=1
        continue
        
    if j_list[i][:14]=="last accessed ":
        h_link+=1
        continue
    if "manuscript" in j_list[i][:15]:
        man+=1
        continue
    if "mimeo" in j_list[i]:
        mim+=1
        continue

in ' advances in econometrics: cointegration, spurious regression, and unit roots,
in (
in /
in [dentification and inference for econometric models: essays in honor of thomas rothenberg, edited by donald w. k. andrews and james h. stock, chap.
in a abertura comercial brasileira nos anos
in a festschrift for erich l. lehmann in honor of his
in a festschrift in honour of david dodge's contributions to canadian public policy,
in a festschrift in honour of david dodge's contributions to canadian public policy:
in a financial history of the netherlands, ed. marjolein 't. hart, joost jonker, and jan luiten van zanden,
in a future of lousy jobs?, ed. by g. burtless. washington, d.c.: brookings institution,
in a game theoretic approach to political economy,
in a nation at war, ed. peter edwards,
in a new economic history of argentina, ed. g. della paolera and a. m. taylor,
in a stream of windows: unsettling reflections on trade, immigration and democracy,
in a. ashenfelter and d. card (eds.) h

In [298]:
print(book_chaps)
print(h_link)
print(man)
print(mim)

3707
62
71
197


In [277]:
rep_names={"a.e.r.":['aea papers and proceedings',
 'aea papers and proceedings,',
 'aea papers and proceedings, american economic review,',
 'a merican economic review',
 'aer papers and proc.',
 'aer papers and proceedings,',
 'aer,',
 'aer.',
 'amer. econ. rev. proc., feb.',
 'amer. econ. rev. proc., may',
 'amer. econ. rev., dec.',
 'amer. econ. rev., june',
 'amer. econ. rev., mar.',
 'amer. econ. rev., mar.,',
 'amer. econ. rev., sept.',
'ameri-can economic review',
'americal economic review,',
 "american 'economic review,",
'american eco-homic review,',
 'american ecomonomic review,',
 'american econ. assoc. papers and proc.',
'american econmomic review,',
 'american econoimc review. https://doi.org/',
 'american econometric review,',
 'american economic association [publisher] inter-university constortium for political and social research [distributor]. https://doi.org/',
 'american economic association [publisher], inter-university consortium for political and social research [distributor]. https:// doi.org/',
 'american economic association [publisher], inter-university consortium for political and social research [distributor]. https://doi.org/',
 'american economic association [publisher]. https://www.ddorn.net/data.htm. last accessed february',
 'american economic association paper and proceedings,',
 'american economic association papers and proceedings,',
 'american economic association, papers and proceedings,',
 'american economic association. https://doi.org/',
'american economic review',
 'american economic review (',
 'american economic review (forthcoming). [',
 'american economic review (p&p),',
 'american economic review (papers & proceedings),',
 'american economic review (papers and proceedings)',
 'american economic review (papers and proceedings),',
 'american economic review (papers and proceedings).',
 'american economic review ,',
 'american economic review p&p,',
 'american economic review papers & proceedings,',
 'american economic review papers and proceeding,',
 'american economic review papers and proceedings,',
 'american economic review papers and proceedings, may,',
 'american economic review proceedings,',
 'american economic review proceedings, may',
 'american economic review,',
 'american economic review, (papers and proceedings),',
 'american economic review, (proceedings),',
 'american economic review, aea papers and proceedings,',
 'american economic review, american economic association,',
 'american economic review, december',
 'american economic review, forthcoming. [',
 'american economic review, june',
 'american economic review, lv,',
 'american economic review, march',
 'american economic review, may',
 'american economic review, no.',
 'american economic review, p&p,',
 'american economic review, paper and proceedings,',
 'american economic review, papers & proceedings,',
 'american economic review, papers and proceedings of the hundred and second annual meeting of the american economic association,',
'american economic review, papers and proceedings,',
 'american economic review, papers and proceedings, may',
 'american economic review, proceedings,',
 'american economic review, september',
 'american economic review,,',
 'american economic review.',
 'american economic review.  https://doi.org/',
 'american economic review. (papers and proceedings),',
 'american economic review. http:// dx.doi.org/',
 'american economic review. http://(www.columbia.edu/~dew',
 'american economic review. http://doi.org/',
 'american economic review. http://dx.doi. org/',
 'american economic review. http://dx.doi.org/',
 'american economic review. http://economics.mit.edu/faculty/acemoglu/data/ajr',
 'american economic review. http://www. aeaweb.org/articles.php?doi=',
 'american economic review. http://www.aeaweb.org/aer/data/june',
 'american economic review. http://www.aeaweb.org/articles.php?doi=',
 'american economic review. http:/dx.doi.org/',
 'american economic review. https://',
 'american economic review. https:// doi.org/',
 'american economic review. https://assets.aeaweb.org/assets/production/articles-attachments/aer/contents/corrigenda/corr_aer.',
 'american economic review. https://doi. org/',
 'american economic review. https://doi.org/',
 'american economic review: aea papers and proceedings,',
 'american economic review: insights',
 'american economic review: insights,',
 'american economic review: papers & proceedings,',
 'american economic review: papers and proceedings,',
 'american economic. review',
 'american economic. review,',
 'american economics journal: applied economics',
 'american economics journal: microeconomics,',
 'american economics review',
 'american economics review,',
 'american economics review, papers and proceedings,',
'american ~ economic review,',
 'american. economic review',
 'american\\neconomic review,',
                     'in american economic review',
 'in american economic review,',
'the american economic review',
 'the american economic review p&p,',
 'the american economic review papers and proceedings,',
 'the american economic review september',
 'the american economic review,',
 'the american economic review, papers and proceedings',
 'the american economic review: papers and proceedings,',
                    ],
           "econometrica":[
               '\\n\\neconometrica,',
               '\\neconometrica,',
               'econ-ometrica',
               'econmetrica',
 'econmetrica,',
 'econo-_metrica,',
 'econo-metrica',
 'econo-metrica,',
 'econo-metrica, march',
 'economeirica,',
 'economerrica,',
 'economertrica,',
 'economet-rica,',
               'econometria,',
               'econometrica',
 "econometrica '",
 'econometrica (',
 'econometrica (forthcoming). [',
 'econometrica ,',
 'econometrica supplemental material,',
 'econometrica supplemental material, http://dx.doi.org/',
 'econometrica supplemental material.',
 'econometrica supplemental materials,',
 'econometrica supplementary material,',
 'econometrica supplementary material, http://www.econometricsociety.org/ecta/supmat/',
 'econometrica supplementary materials,',
"econometrica'",
 'econometrica,',
 'econometrica, $',
 'econometrica, ,',
 'econometrica, april',
 'econometrica, april,',
 'econometrica, available at http://www.econometricsociety.org/ecta/supmat.',
 'econometrica, forthcoming,',
 'econometrica, january',
 'econometrica, july',
 'econometrica, lx,',
 'econometrica, nov.',
 'econometrica, november',
 'econometrica, october,',
 'econometrica, sept.',
 'econometrica, september',
 'econometrica, so,',
 'econometrica, t',
 'econometrica, xl,',
 'econometrica, xxv,',
 'econometrica, xxxi,',
 'econometrica, §',
 'econometrica, §§,',
 'econometrica,,',
 'econometrica,y',
 'econometrica.',
 'econometrica.[',
 'econometrica: journal of the econometric society,',
 'econometrica;',
                'forthcoming in econometrica,',
               'in econometrica supplemental material,',
               
               
               
           ],
    "j.p.e.":[
        '[pe',
 '[pe.',
     '[. pe.',
 '[.-pe.',
 '[.p-e.',
 '[.p.e.',
 '[.pe.',
 '[/ pe.',
'[ pe.',
 '[,pe.',
 '[-p.e.',
 '[-pe.',
    '/pe.',
    '/[.p.e.',
     '/.p-e.',
 '/.p.e.',
 '/.pe',
 '/.pe.',
'/.-p.e.',
 '/.-pe.',
    '/.,pe.',
    '/. pe.',
     '/-p-e.',
 '/-p.e.',
 '/-pe',
 '/-pe.',
'/, pe.',
 '/,pe.',
    '/ pe.',
        '].pe.',
        'j pe,',
         'j pe.',
        'j,pe.',
 'j-p-e.',
 'j-p.e.',
 'j-p_e.',
 'j-pe.',
'j. pe.',
        'j. polit. econ.,',
 'j. polit. econ., apr.',
 'j. polit. econ., dec.',
 'j. polit. econ., feb.',
 'j. polit. econ., july/aug.',
 'j. polit. econ., june',
 'j. polit. econ., mar./apr.',
 'j. polit. econ., mar./apr.,',
 'j. polit. econ., may/june',
 'j. polit. econ., nov.',
 'j. polit. econ., oct.',
 'j. polit. econ., sept./oct.',
 'j. political econ.',
 'j. political economy,',
 'j.-e.',
 'j.-pe.',
 'j.-re.',
 'j./-e.',
 'j.p-e.',
 'j.p.',
 'j.p.e.',
 'j.p.e..',
        'j.pe.',
        'j:pe.',
        'jle.',
        'journal of polical economy,',
 'journal of policital economy,',
        'journal of politcal economy,',
 'journal of political econ-omy,',
 'journal of political econmy,',
 'journal of political economics,',
 'journal of political economy',
 'journal of political economy (forthcoming). [',
 'journal of political economy ,',
 'journal of political economy,',
 'journal of political economy, \\',
 'journal of political economy, \\\\',
 'journal of political economy, april',
 'journal of political economy, august',
 'journal of political economy, cvi (',
 'journal of political economy, december',
 'journal of political economy, february',
 'journal of political economy, iic, $',
 'journal of political economy, july',
 'journal of political economy, june',
 'journal of political economy, lvi,',
 'journal of political economy, lxvi, no.',
 'journal of political economy, lxxv,',
 'journal of political economy, lxxxi (july/aug.',
 'journal of political economy, lxxxii (march/april,',
 'journal of political economy, lxxxv (',
 'journal of political economy, lxxxvi (march/april',
 'journal of political economy, march-april',
 'journal of political economy, october',
 'journal of political economy, supp.',
 'journal of political economy, xcix,',
 'journal of political economy,,',
 'journal of political economy,\\n',
 'journal of political economy.',
 'journal of political econonomy,',
        'journal of political,',
 'journal of political\\neconomy,',
 'journal of politicial economy,',
        'jp.',
 'jpe,',
 'jpe.',
         'journal\\nof political economy,',
         'journal political economy,',
        'p-e.',
         'the journal of political economy,',
        '| pe.',
 '|. pe.',
        
    ],
           "q.j.e.":['q, j.e.',
 'q.',
 'q. (-e.',
 'q. (ef.',
 'q. -e.',
 'q. .e.',
 'q. /-e.',
 'q. /-f.',
 'q. /-£.',
 'q. /.e.',
 'q. /e.',
 'q. [-e.',
 'q. e.',
 'q. j-e.',
'q. j. e.', 
'q. j.-e.',
 'q. j.e.',
 'q. je.',
'q.-e.',
 'q./-e',
 'q./-e.',
 'q./-£.',
 'q./. e.',
 'q./.-e.',
 'q./.e.',
 'q./.£.',
 'q./e.',
 'q.[-e.',
 'q.[.e.',
 'q.]-e.',
 'q.].e.',
 'q.f-e.',
 'q.f.e.',
 'q.j-e.',
 'q.j-e. electronically published may',
 'q.j.-e.',
 'q.j.e.',
 'q.j.£.',
 'q.j/-e',
 'q.jj.e.',
 'qj.',
 'qj.e.',
 'qje.',
                     'quar-terly journal of economics,',
 'quaraterly journal of economics,',
 'quarlery journal of economics,',
 'quart. j. econ.,',
 'quart. j. econ., aug.',
 'quart. j. econ., feb.',
 'quart. j. econ., may',
 'quarterly jour-nal of economics',
 'quarterly journal economics,',
'quarterly journal of eco-nomics',
 'quarterly journal of eco-nomics,',
 'quarterly journal of econimics,',
 'quarterly journal of economic',
 'quarterly journal of economic,',
 'quarterly journal of economics',
 'quarterly journal of economics (forthcoming). [',
 'quarterly journal of economics (supplement),',
 'quarterly journal of economics |',
 'quarterly journal of economics,',
 'quarterly journal of economics, (',
 'quarterly journal of economics, \\\\',
 'quarterly journal of economics, august',
 'quarterly journal of economics, august,',
 'quarterly journal of economics, cii,',
 'quarterly journal of economics, cix,',
 'quarterly journal of economics, cv,',
 'quarterly journal of economics, cv-',
 'quarterly journal of economics, cviii,',
 'quarterly journal of economics, cxiii,',
 'quarterly journal of economics, cxvi,',
 'quarterly journal of economics, cxxi,',
 'quarterly journal of economics, cyx,',
 'quarterly journal of economics, feb.,',
 'quarterly journal of economics, february',
 'quarterly journal of economics, february,',
 'quarterly journal of economics, lxxv,',
 'quarterly journal of economics, lxxxvii,',
 'quarterly journal of economics, march',
 'quarterly journal of economics, may',
 'quarterly journal of economics, november',
 'quarterly journal of economics, november,',
 'quarterly journal of economics, supplement,',
 'quarterly journal of economics,,',
 'quarterly journal of economics.',
 'quarterly journal of economics. [',
                      'quarterly journal of\\neconomics,',
                     'the quartely journal of economics,',
 'the quarterly journal of economics',
 'the quarterly journal of economics,',
                     'the quaterly journal of economics,',
                     
    
],
"r.e.s.":['rev. econ studies',
          'rev. econ. stud.',
 'rev. econ. stud. july',
 'rev. econ. stud., apr.',
 'rev. econ. stud., jan.',
 'rev. econ. studies',
 "rev. econ. studies '",
 'rev. econ. studies, forthcoming. https://doi.org/',
 'rev. econ. studies. doi:',
 'rev. econ. studies. electronically published june',
 'rev. econonomic studies',
          'reveiw of economic studies,',
          'review of econ-omic studies,',
         'review of economic studies',
 'review of economic studies (forthcoming). [',
 'review of economic studies (symposium on the economics of exhaustible resources),',
 'review of economic studies (symposium),',
 'review of economic studies (this issue),',
 'review of economic studies symposium (february),',
 'review of economic studies symposium, (february),',
 'review of economic studies,',
 'review of economic studies, doi:',
 'review of economic studies, forthcoming. [',
 'review of economic studies, january',
 'review of economic studies, july',
 'review of economic studies, june',
 'review of economic studies, lv,',
 'review of economic studies, october',
 'review of economic studies, symposium on the economics of exhaustible resources,',
 'review of economic studies, symposium,',
 'review of economic studies, xlviii (',
 'review of economic studies, xxiii,',
 'review of economic studies, §',
 'review of economic studies,,',
 'review of economic studies,\\n\\n',
 'review of economic studies. [',
 'review of economic studies. symposium (',
 'review of economic\\nstudies,',
 'review of economical studies,',
          'review of economics and studies,',
           'review of economics studies',
 'review of economics studies,',
          'review of u economic studies,',
          'review, of economic studies,',
 'review-of economic studies,',
 'review\\nof economic studies,',
 'reviews of economic studies,',
          'the review of economic studies',
 'the review of economic studies,',
 'the review of economic studies, doi:',
 'the review of economic studies, rdw',
 'the review of economic studies. [',
          'the review of economics studies,',
          'the review of\\n\\neconomic studies,',
          '\\n\\nreview of economic studies',
          '\\nreview of economic studies,',
                    ]
           
        }



In [278]:

j_list=list(set(j_list))
j_list.sort()
len(j_list)

14501

In [288]:
'\\n' in j_list[105][:3]


True

In [296]:
j_list[9999:]

['medical education,',
 'medical hypotheses',
 'medical journal of australia',
 'medicinsk forum',
 'meds discussion paper no.',
 'meeting papers no.',
 'melbourne business school working paper number',
 'memo',
 'memo. no.',
 'memoirs of the american mathematical society,',
 'memorandum no.',
 'memorandum prepared for a conference at cambridge, england, july',
 'memorandum prepared for a conference in cambridge, england, july',
 'memorandum, institute of economics, university of oslo,',
 'memory & cognition,',
 'memory and cognition,',
 'memory,',
 'merrika,',
 'messenger of mathematics,',
 'meteor research memorandum rm/',
 'methodology and computing in applied probability,',
 'methodology and science,',
 'methods and applications of analysis,',
 'methods of operations research,',
 'metrika,',
 'metroeconomica',
 'metroeconomica,',
 'metroeconomica, viii:',
 'metron',
 'metron,',
 'michigan law rev.',
 'michigan law review',
 'michigan law review,',
 'michigan retirement research cen

In [229]:
set(keys)

{'authors', 'full_reference', 'month', 'pages', 'publisher', 'title', 'year'}

In [299]:
p="altinoglu, levent. na. the origins of aggregate fluctuations in a credit network economy. journal of monetary economics, forthcoming."
p='ACEMOGLU, DARON, VASCO M. CARVALHO, ASUMAN OZDAGLAR, AND ALIREZA TAHBAZ-SALEHI. 2012. THE NETWORK ORIGINS OF AGGREGATE FLUCTUATIONS. ECONOMETRICA, 80: 1977-2016.'
p='dupuy, a., a. galichon, s. jaffe, and s. d. kominers (2017): "taxation in matching markets," available at ssrn, https://papers.ssrn.com/sol3/papers.cfm ?abstract_id=3060746. .1636,1639.'
q="the origins of aggregate fluctuations in a credit network economy"
r='RECENT TRENDS IN INSURED AND UNEMPLOYED UNEMPLOYMENT: IS THERE AN EXPLANATION?'
print(q in p)
print(r in p)
temp2=re.search(r'(\D+?.)((\d+)|na).([^.]+).(.+), (\d+): (.+)',p)
temp2=re.search(r'(\D+).(\d+[^0-9.]*?|na).([^.]*).(.+?)(\d+):(.*)', p)
temp2=re.search(r'([^()":]*)(\(\d+?\D*?\)|)([:. ]*)"(.+)" (available.*?)(https.*)',p)
temp2=re.search(r'([^()":]*)(\(\d+?\D*?\)|)([:. ]*)"(.+)"([^"]+)(vol.*|\d.*)',p)
temp2.groups()

False
False


('dupuy, a., a. galichon, s. jaffe, and s. d. kominers ',
 '(2017)',
 ': ',
 'taxation in matching markets,',
 ' available at ssrn, https://papers.ssrn.com/sol3/papers.cfm ?abstract_id=3060746. .1636,163',
 '9.')

In [405]:
re.search('(intangible capital and u.s. economic growth){e<=5}','corrado, c.; c. hulten; d. sichel (2009): "intangible capital and u.s.," economic growth. review of income and wealth, 55 (3), 661-685. [2619,2621]')

In [231]:
t='MANUSCRIPT RECEIVED 9 NOVEMBER, 2017; FINAL VERSION ACCEPTED 20 APRIL, 2019; AVAILABLE ONLINE 3 MAY, 2019'
re.search(r"MANUSCRIPT RECEIVED.*FINAL VERSION ACCEPTED",t)

<re.Match object; span=(0, 60), match='MANUSCRIPT RECEIVED 9 NOVEMBER, 2017; FINAL VERSI>

In [232]:
manu2.sort()
manu2

[]

In [233]:
manu.sort()
manu

[]

In [234]:
temp='ANDREWS, I.; M. GENTZKOW; J. M. SHAPIRO (2017): “MEASURING THE SENSITIVITY OF PARAMETER ESTIMATES TO ESTIMATION MOMENTS,” QUARTERLY JOURNAL OF ECONOMICS, 132 (4), 1553-1592.'
print(re.search(r'([^()]*) \((.*?)\)([:. ]*)"([^()]+)"((\D)+)(.+)',re.sub(r"[“”]",'"',temp)).groups())

('ANDREWS, I.; M. GENTZKOW; J. M. SHAPIRO', '2017', ': ', 'MEASURING THE SENSITIVITY OF PARAMETER ESTIMATES TO ESTIMATION MOMENTS,', ' QUARTERLY JOURNAL OF ECONOMICS, ', ' ', '132 (4), 1553-1592.')


In [235]:
len(references.keys())

1

In [236]:
js_3

0

In [237]:
js_2

0

In [238]:
js

1

In [239]:
av

1

In [240]:
js_4

0

In [241]:
js_5

0

In [242]:
x=open_json(base_path+"48628742_chatgpt.json")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sijiawu/Work/Refs Danae/Thesis/Data/48628742_chatgpt.json'

In [243]:
with open(file) as f:
    df = json.load(f)

NameError: name 'file' is not defined

In [ ]:
CFV        